In [5]:
import pandas as pd
import pywt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

df = pd.read_csv("sample_data/user_0001/user_0001_highway.csv", index_col=0)

def preprocess(df):
    return (
        df.drop(
            ["FOG",
             "FOG_LIGHTS",
             "FRONT_WIPERS",
             "HEAD_LIGHTS",
             "RAIN",
             "REAR_WIPERS",
             "SNOW",
            ], axis=1
        )
    )


test = preprocess(df)
test

,ACCELERATION,ACCELERATION_PEDAL,ACCELERATION_Y,ACCELERATION_Z,BRAKE_PEDAL,CLUTCH_PEDAL,CURVE_RADIUS,DISTANCE,DISTANCE_TO_NEXT_INTERSECTION,DISTANCE_TO_NEXT_STOP_SIGNAL,...,LANE_LATERAL_SHIFT_LEFT,LANE_LATERAL_SHIFT_RIGHT,LANE_WIDTH,ROAD_ANGLE,SPEED,SPEED_LIMIT,SPEED_NEXT_VEHICLE,SPEED_Y,SPEED_Z,STEERING_WHEEL
0,0.229616,0.000000,-0.016220,0.253225,400.0,1.0,111.88820,0.007735,472.7118,99000.0,...,-1.481436,2.311564,5.5,99.55229,0.013202,90.0,0.0,-0.002931,-0.057344,-0.003260
1,0.229616,0.000000,-0.016220,0.253225,400.0,1.0,111.88820,0.008466,472.7118,99000.0,...,-1.481436,2.311564,5.5,99.55229,0.013202,90.0,0.0,-0.002931,-0.057344,-0.003260
2,0.229616,0.000000,-0.016220,0.253225,400.0,1.0,111.88820,0.008466,472.7118,99000.0,...,-1.481436,2.311564,5.5,99.55229,0.013202,90.0,0.0,-0.002931,-0.057344,-0.003260
3,-0.197076,0.000000,0.027126,0.113095,400.0,1.0,111.88820,0.009322,472.7118,99000.0,...,-1.481436,2.311564,5.5,99.55229,0.011758,90.0,0.0,-0.001833,-0.026665,-0.004027
4,-0.188414,0.000000,0.028093,0.055052,400.0,1.0,111.88820,0.009529,472.7118,99000.0,...,-1.481436,2.311564,5.5,99.55229,0.008247,90.0,0.0,-0.001333,-0.025210,-0.004219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.249120,0.436537,0.909279,-0.053026,0.0,0.0,52.33913,11.246339,461.3564,99000.0,...,-0.828703,2.964297,5.5,106.84390,7.716069,90.0,0.0,0.122129,-0.128525,0.711598
996,2.222887,0.434193,0.917394,-0.055644,0.0,0.0,52.33913,11.327696,461.3564,99000.0,...,-0.828703,2.964297,5.5,106.84390,7.734049,90.0,0.0,0.122830,-0.128602,0.715241
997,2.187932,0.431263,0.930866,-0.052662,0.0,0.0,50.53785,11.405407,461.1497,99000.0,...,-0.822391,2.970609,5.5,107.07400,7.770628,90.0,0.0,0.124259,-0.128696,0.721569
998,2.187932,0.431263,0.930866,-0.052662,0.0,0.0,50.53785,11.483337,461.1497,99000.0,...,-0.822391,2.970609,5.5,107.07400,7.770628,90.0,0.0,0.124259,-0.128696,0.721569


In [3]:
from torch.utils.data import DataLoader, TensorDataset

def get_dataset(x):
    return TensorDataset(torch.from_numpy(x))

get_dataset(test.to_numpy())

## Wavelet transform

In [7]:
def van_haar(data):
    ncol = data.shape[1]
    nrow = data.shape[0]
    for i in range(ncol):
        cur_col = data[:,i].copy()
        (cA, cD) = pywt.dwt(cur_col, 'haar')
        new_col = np.reshape(np.concatenate((cA,cD), 0),(nrow,1))
        data = np.hstack((data,new_col))
    return data

van_haar(test.to_numpy())

array([[ 2.29616400e-01,  0.00000000e+00, -1.62202900e-02, ...,
        -4.14544886e-03, -8.10961251e-02, -4.61006893e-03],
       [ 2.29616400e-01,  0.00000000e+00, -1.62202900e-02, ...,
        -3.36891904e-03, -5.94027608e-02, -5.15243043e-03],
       [ 2.29616400e-01,  0.00000000e+00, -1.62202900e-02, ...,
        -1.63602285e-03, -3.52803787e-02, -5.96597164e-03],
       ...,
       [ 2.18793200e+00,  4.31263100e-01,  9.30865900e-01, ...,
        -9.65837152e-04,  1.46229682e-04, -5.15240427e-03],
       [ 2.18793200e+00,  4.31263100e-01,  9.30865900e-01, ...,
        -1.01010204e-03,  6.61851947e-05, -4.47450100e-03],
       [ 2.16781900e+00,  4.26575500e-01,  9.44088300e-01, ...,
        -1.01575889e-03,  1.20915260e-05, -4.47443029e-03]])

In [8]:
pywt.dwt(test, 'haar')[1].shape

(1000, 16)

## PyTorch

In [9]:
class TemporalBlock(nn.Module):
    def __init__(
            self, 
            n_inputs, 
            n_outputs, 
            kernel_size, 
            stride,
            dilation,
            padding,
            dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(
            nn.Conv2d(
                n_inputs, 
                n_outputs, 
                kernel_size,
                stride=stride, 
                padding=padding,
                dilation=dilation))
        self.pad = torch.nn.ZeroPad2d((padding, 0, 0, 0))
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
        self.conv2 = weight_norm(
            nn.Conv2d(
                n_outputs, 
                n_outputs, 
                kernel_size,
                stride=stride, 
                padding=padding, 
                dilation=dilation))
        self.net = nn.Sequential(
            self.pad, 
            self.conv1, 
            self.relu, 
            self.dropout,
            self.pad, 
            self.conv2, 
            self.relu, 
            self.dropout)
        
        self.downsample = nn.Conv1d(
            n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, 
                                     out_channels, 
                                     kernel_size, 
                                     stride=1, 
                                     dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, 
                                     dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [10]:
class TCN(nn.Module):
    def __init__(
            self, 
            input_size, 
            output_size, 
            num_channels, 
            kernel_size, 
            dropout):
        super(TCN, self).__init__()
        
        self.tcn = TemporalConvNet(input_size, 
                                   num_channels, 
                                   kernel_size=kernel_size, 
                                   dropout=dropout)
        
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in)"""
        y1 = self.tcn(inputs)  # input should have dimension (N, C, L)
        o = self.linear(y1[:, :, -1])
        return o

In [11]:
input_channels = 23
n_classes = 51
channel_sizes = (25,25,25,25,25)
kernel_size = 7
dropout = 0.1
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=dropout)

In [18]:
model.train()

TCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv2d(23, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
        (pad): ZeroPad2d((6, 0, 0, 0))
        (relu): ReLU()
        (dropout): Dropout(p=0.1, inplace=False)
        (conv2): Conv2d(25, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
        (net): Sequential(
          (0): ZeroPad2d((6, 0, 0, 0))
          (1): Conv2d(23, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): ZeroPad2d((6, 0, 0, 0))
          (5): Conv2d(25, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
          (6): ReLU()
          (7): Dropout(p=0.1, inplace=False)
        )
        (downsample): Conv1d(23, 25, kernel_size=(1,), stride=(1,))
      )
      (1): TemporalBlock(
        (conv1): Conv2d(25, 25, kernel_size=(7, 7), stride=(1, 1), padding=(12, 12), dilation=(2, 2))
        (pad): ZeroPad2d

In [19]:
model.eval()

TCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv2d(23, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
        (pad): ZeroPad2d((6, 0, 0, 0))
        (relu): ReLU()
        (dropout): Dropout(p=0.1, inplace=False)
        (conv2): Conv2d(25, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
        (net): Sequential(
          (0): ZeroPad2d((6, 0, 0, 0))
          (1): Conv2d(23, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): ZeroPad2d((6, 0, 0, 0))
          (5): Conv2d(25, 25, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6))
          (6): ReLU()
          (7): Dropout(p=0.1, inplace=False)
        )
        (downsample): Conv1d(23, 25, kernel_size=(1,), stride=(1,))
      )
      (1): TemporalBlock(
        (conv1): Conv2d(25, 25, kernel_size=(7, 7), stride=(1, 1), padding=(12, 12), dilation=(2, 2))
        (pad): ZeroPad2d

In [20]:
model(get_dataset(test.to_numpy()))

AttributeError: 'TensorDataset' object has no attribute 'dim'